In [1]:
# best results when running `ocean events access`
import datetime, time, json, requests, logging

from squid_py.config import Config
from squid_py.ocean.ocean import Ocean
from ocean_cli.ocean import get_default_account
from ocean_cli.api.assets import (
    list_assets, 
    publish, 
    authorize,
    consume,
    search
)

logging.getLogger().setLevel(logging.ERROR)

Using default logging settings.


In [2]:
config_bob = Config(filename='./bob.ini')
ocean = Ocean(config_bob)
bob = get_default_account(config_bob)
ocean.tokens.request(bob, 1)
print(ocean.accounts.balance(bob))

Balance(eth=1000000000000000242842900000, ocn=1999999999999999998)


In [3]:
config_alice = Config(filename='./alice.ini')
ocean_alice = Ocean(config_alice)
alice = get_default_account(config_alice)
print(ocean.accounts.balance(alice))

Balance(eth=999999999999999999168430000, ocn=1000000000000000000)


In [4]:
# publish url
did = publish(ocean, bob, name='put', secret='https://i.giphy.com/media/3oEduQAsYcJKQH2XsI/giphy.webp', price=0)

# consume did
from IPython.display import Image
Image(url=authorize(ocean, bob, did)[2])

Not able to decrypt


AttributeError: 'str' object has no attribute 'get'

In [ ]:
# list last 10 assets
assets = list_assets(ocean, bob, None)
print(assets[-10:])

In [ ]:
# search assets for text
print(search(ocean, 'img', pretty=True))

In [ ]:
# resolve did in ddo
print(ocean.assets.resolve(assets[-1]).as_dictionary())

In [ ]:
# publish url with json
url = 'https://api.coingecko.com/api/v3/simple/price?ids=ocean-protocol&vs_currencies=EUR%2CUSD'
did_json = publish(ocean, bob, name='json', secret=url, price=2)

# decrypt url & consume json
decrypted_url = authorize(ocean, bob, did_json)[2]
print(json.dumps(requests.get(decrypted_url).json(), indent=2, sort_keys=True))

In [ ]:
# publish url with json
url = 'https://api.giphy.com/v1/gifs/random?api_key=0UTRbFtkMxAplrohufYco5IY74U8hOes&tag=fail&rating=pg-13'
did_random = publish(ocean, bob, name='img', secret=url, price=len(url))

# decrypt url, resolve payload
decrypted_url = authorize(ocean, bob, did_random)[2]
try:
    img = Image(url=requests.get(decrypted_url).json()['data']['images']['original']['url'])
    display(img)
except TypeError as e:
    print('pass / api throttle')

In [ ]:
# publish csv
url = 'https://raw.githubusercontent.com/plotly/datasets/master/school_earnings.csv'
did_csv = publish(ocean, bob, name='csv', secret=url, price=len(url))

# consume csv
decrypted_url = authorize(ocean, bob, did_csv)[2]
import pandas as pd
pd.read_csv(decrypted_url).describe()

In [ ]:
# serve files from localhost
# run `python -m http.server` and encrypt file name
did_localhost = publish(ocean, bob, name='readme', secret='README.md', price=0, 
                        service_endpoint='http://localhost:8000')

# order and consume request
response = consume(ocean, bob, did_localhost, *authorize(ocean, bob, did_localhost), method='api')
print(response.text)

In [ ]:
# publish proxy api with token
# run `python proxy.py` and encrypt api token
url = 'http://localhost:8080'
token = 'muchsecrettoken'
did_api = publish(ocean, bob, name='api', 
                  secret=[{'url': 'run', 'token': token}], 
                  price=len(url), 
                  service_endpoint=url)

# consume api with token
print(consume(ocean, bob, did_api, *authorize(ocean, bob, did_api), method='api').json())

In [ ]:
from ocean_cli.api.notebook import create_notebook
print(create_notebook(did, did))

In [ ]:
from ocean_cli.api.assets import authorize
response = authorize(ocean, bob, 'did:op:41ee01268e124bc3a539e2477e5dd4abf46c7395371942a3aea2a73851837b4d')
response

In [ ]:
Image(url=response[2])

In [ ]:
from ocean_cli.api.assets import authorize
response = authorize(ocean, bob, 'did:op:ea1b821b4270489db73b836419da5c9e22c83272bed34e7ba54f518100e46ce7')
response

In [ ]:
Image(url=response[2])

In [ ]:
from ocean_cli.api.assets import authorize
response = authorize(ocean, bob, 'did:op:3d756d6264ba4513baaea36416ff2f4e579279ce642c4347b2fc8d9151145d62')
response

In [ ]:
df = pd.read_csv(decrypted_url)
df